In [116]:
from bs4 import BeautifulSoup
import requests as requests
import pandas as pd
from tqdm import tqdm

In [118]:
# Create a list of the items to scrape
n_posts = range(1, 10)
df = pd.DataFrame(columns=['item','title', 'n_comments','corpus' ,'boost', 'sats', 'betha','commentors','external_links'])
for number in tqdm(n_posts):
    # Initialize the dataframe
 
    url_posts = f'https://stacker.news/items/{number}'
    response = requests.get(url_posts)
    soup = BeautifulSoup(response.text, 'html.parser')
    index = number-1
#-------------------------------------------------------------------------------
    # TITLE
    try:
        title = soup.find('a', class_='item_title__FH7AS text-reset me-2').get_text()
    except:
        continue
    # add title to dataframe
    df.at[index, 'title'] = title
#-------------------------------------------------------------------------------
    # ITEM
    df.at[index, 'item'] = number
#-------------------------------------------------------------------------------
    # CORPUS
    try:
        corpus=soup.find('div', class_="item_fullItemContainer__ZAYtZ").get_text()
    except:
        df.at[index, 'corpus'] = "NaN"
    # add title to dataframe
    df.at[index, 'corpus'] = corpus
#-------------------------------------------------------------------------------
    # EXTERNAL LINKS
    try:
        ex_link=soup.find('a', class_="item_link__4cWVs").get_text()
    except:
        df.at[index, 'external_links'] = "NaN"
    # add external links to dataframe
    df.at[index, 'external_links'] = ex_link

#-------------------------------------------------------------------------------
    # BANNER
    try:
        banner = soup.find('div', class_='item_other__MjgP3')
    except:
        df.at[index, 'boost'] = "NaN"
        df.at[index, 'sats'] = "NaN"
        df.at[index, 'betha'] = "NaN"
    # deal with banner
    banner_data = []
    for i in banner.find_all('span'):
        banner_data.append(i.text)
    for b in banner_data:
        if "boost" in b:
            df.at[index, 'boost'] = b
        if "sats" in b:
            df.at[index, 'sats'] = b
        if "@" in b:
            df.at[index, 'betha'] = b
#-------------------------------------------------------------------------------
    # N_COMMENTS
    try:
        n_comments = soup.find('a', class_='text-reset position-relative').get_text()
    except:        
        df.at[index, 'n_comments'] = "NaN"
    # add n_comments to dataframe
    df.at[index, 'n_comments'] = n_comments
#-------------------------------------------------------------------------------
    # COMMENTORS
    
    a_elements = soup.find_all('a')
    
    df.at[index, 'commentors'] = "NaN"
    at_elements = []

    for el in a_elements:
        links = el.get_text() 
        if links.startswith('@'):
            at_elements.append(el)
    
    commentors_list=[]
    for ind in range(0,len(at_elements)):
        commentors_list.append(at_elements[ind]["href"])
    # add commentors to dataframe
    df.at[index, 'commentors'] = commentors_list
#-------------------------------------------------------------------------------
# Fixing the dataframe
df['author'] = df['betha'].str.extract(r'@(\w+)')
df['data'] = df['betha'].str.extract(r'(\d+\s\w+\s\d+)')
df.drop('betha', axis=1, inplace=True)
df

100%|██████████| 9/9 [00:07<00:00,  1.25it/s]


,item,title,n_comments,corpus,boost,sats,commentors,external_links,author,data
0,1,El Salvador Makes Bitcoin Legal Tender,13 comments,,4 boost,603 sats,"[/k00b, /ln123, /k00b, /k00b, /OneOneSeven, /e...",bitcoinmagazine.com/business/el-salvador-appro...,k00b,11 Jun 2021
1,2,stacker news soft launch AMA,30 comments,Figured I'd make a place to raise questions ab...,NaN,2 sats,"[/k00b, /sha256, /k00b, /banana, /banana, /k00...",bitcoinmagazine.com/business/el-salvador-appro...,k00b,11 Jun 2021
7,8,The Lightning Conference 2019: Alex Bosworth,2 comments,,NaN,0 sats,"[/k00b, /k00b, /k00b, /k00b, /kr, /ekzyis]",www.youtube.com/watch?v=Y4uxMtFwaYc,k00b,11 Jun 2021


# Function pandas.explode

The following function explodes the selected dataframe by the desired column. This helps us to visualize in each row the relationship between the post item and the comment item, and therefore let us merge the dataframe with itself in order to retrieve the relationship between the users and all the details of the item (sats, replies etc) 

In [112]:
new_df = df.explode('commentors')

In [113]:
new_df

,item,title,n_comments,corpus,boost,sats,commentors,external_links,author,data
0,1,El Salvador Makes Bitcoin Legal Tender,13 comments,,4 boost,603 sats,/k00b,bitcoinmagazine.com/business/el-salvador-appro...,k00b,11 Jun 2021
0,1,El Salvador Makes Bitcoin Legal Tender,13 comments,,4 boost,603 sats,/ln123,bitcoinmagazine.com/business/el-salvador-appro...,k00b,11 Jun 2021
0,1,El Salvador Makes Bitcoin Legal Tender,13 comments,,4 boost,603 sats,/k00b,bitcoinmagazine.com/business/el-salvador-appro...,k00b,11 Jun 2021
0,1,El Salvador Makes Bitcoin Legal Tender,13 comments,,4 boost,603 sats,/k00b,bitcoinmagazine.com/business/el-salvador-appro...,k00b,11 Jun 2021
0,1,El Salvador Makes Bitcoin Legal Tender,13 comments,,4 boost,603 sats,/OneOneSeven,bitcoinmagazine.com/business/el-salvador-appro...,k00b,11 Jun 2021
0,1,El Salvador Makes Bitcoin Legal Tender,13 comments,,4 boost,603 sats,/ekzyis,bitcoinmagazine.com/business/el-salvador-appro...,k00b,11 Jun 2021
0,1,El Salvador Makes Bitcoin Legal Tender,13 comments,,4 boost,603 sats,/k00b,bitcoinmagazine.com/business/el-salvador-appro...,k00b,11 Jun 2021
0,1,El Salvador Makes Bitcoin Legal Tender,13 comments,,4 boost,603 sats,/ekzyis,bitcoinmagazine.com/business/el-salvador-appro...,k00b,11 Jun 2021
0,1,El Salvador Makes Bitcoin Legal Tender,13 comments,,4 boost,603 sats,/Bitman,bitcoinmagazine.com/business/el-salvador-appro...,k00b,11 Jun 2021
0,1,El Salvador Makes Bitcoin Legal Tender,13 comments,,4 boost,603 sats,/ekzyis,bitcoinmagazine.com/business/el-salvador-appro...,k00b,11 Jun 2021
